In [ ]:
"""
Explanation of the Workflow:
User Query: The bot receives a question from the user as input (query).

Embed the Query: The query is converted into a vector embedding using OpenAI’s text-embedding-ada-002 
model (high-dimensional representation of the text).

Retrieve Relevant Documents: This query embedding is used to search the Pinecone vector database. Pinecone 
returns the top 5 most similar documents from its index based on the similarity between their embeddings
and the query embedding.

Generate Context: The retrieved documents’ content is concatenated into a context, which will help the GPT model
generate an informed answer.

Generate Final Answer: OpenAI GPT is then given both the original query and the context to produce a 
well-informed answer.

Return the Answer: Finally, the answer generated by GPT is returned to the user.

This pseudocode simplifies the entire RAG model into clear, sequential steps.
"""


In [2]:
!pip install pinecone-client==5.0.1 openai==0.28.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.9 MB/s eta 0:00:00


In [3]:
from pinecone import Pinecone
import openai

OPENAI_API_KEY="your_key"
openai.api_key = OPENAI_API_KEY

PINECONE_API_KEY = "your_key"
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("medical-docs") # NOTE: make sure this exist on the server. Dimension = 1536

In [4]:
###################### Define Functions for Embedding and Document Retrieval
# Create a function to generate embeddings using OpenAI's embedding model, and a
# function to retrieve relevant documents using Pinecone.
# Function to get embedding from OpenAI
def get_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response['data'][0]['embedding']
###########################

# Function to retrieve relevant documents from Pinecone
def retrieve_documents(query, top_k=1):
    query_embedding = get_embedding(query)

    query_results = index.query(
    vector=query_embedding,
    top_k=1,
    include_metadata=True  # Include metadata in the results to retrieve the text
    )

    documents = []
    for match in query_results['matches']:
        documents.append(match)

    return documents

In [5]:
########################### Insert Documents into Pinecone Index
# Here’s how to insert (upsert) documents into the Pinecone
# index. Each document is converted into an embedding and stored with a unique ID.
# Sample documents to insert
documents = [
    {"id": "doc1", "text": "Medical document content for Doc 1. Side effects of Aspirin are headache, and ulcers"},
    {"id": "doc2", "text": "Medical document content for Doc 2. Medications should be taken under supervision of doctors"},
]

# Insert documents into the Pinecone index
for doc in documents:
    embedding = get_embedding(doc['text'])  # Get embedding from OpenAI model
    print("len(embedding)=", len(embedding))  # Should print 1536
    index.upsert([{
        'id': doc['id'],
        'values': embedding,
        'metadata': {'text': doc['text']}  # Include document text as metadata
    }])

print("Medical documents have been inserted into the index.")


len(embedding)= 1536
len(embedding)= 1536
Medical documents have been inserted into the index.


In [6]:
#######################Generate Answer with GPT-4
# Now, retrieve the most relevant documents from Pinecone based on a query
# and generate an answer using  GPT-4.
# Function to generate answer using OpenAI GPT
def generate_answer(query, context):
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"

    response = openai.ChatCompletion.create(
        model="gpt-4",  # or "gpt-3.5-turbo"
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=150
    )

    return response["choices"][0]["message"]["content"]

# Function to get final answer from RAG model
def rag_qa_bot(query):
    # Retrieve relevant documents from Pinecone
    retrieved_docs = retrieve_documents(query)
    print("retrieved_docs=",retrieved_docs)
    print("len(retrieved_docs)=",len(retrieved_docs))
    # Prepare context by concatenating the retrieved document texts
    context = "\n".join([doc['metadata']['text'] for doc in retrieved_docs])
    print("context=",context)

    # Generate the final answer using OpenAI GPT
    final_answer = generate_answer(query, context)
    return final_answer

In [7]:
#################################Test the QA Bot
# Finally, test the RAG-based QA bot with a sample question.
query = "What are the side effects of taking Aspirin?"
answer = rag_qa_bot(query)

print("Answer:", answer)

retrieved_docs= [{'id': 'doc1',
 'metadata': {'text': 'Medical document content for Doc 1. Side effects of '
                      'Aspirin are headache, and ulcers'},
 'score': 0.898047745,
 'values': []}]
len(retrieved_docs)= 1
context= Medical document content for Doc 1. Side effects of Aspirin are headache, and ulcers
Answer: The side effects of taking Aspirin are headache, and ulcers.
